In [77]:
from random import uniform
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

WORLD_SIZE = 100.0
MEASUREMENT_RANGE = 100
MOTION_NOISE = 1.0
MEASUREMENT_NOISE = 2.0
N_LANDMARKS = 10
LANDMARKS = np.array(
    [[uniform(0, WORLD_SIZE), uniform(0, WORLD_SIZE)]
     for _ in range(N_LANDMARKS)]
    )
display(WORLD_SIZE)
display(MEASUREMENT_RANGE)
display(MOTION_NOISE)
display(MEASUREMENT_NOISE)
display(LANDMARKS)

100.0

100

1.0

2.0

array([[58.55207768, 32.62117737],
       [77.25132288, 72.2690247 ],
       [94.40584323, 88.01817065],
       [13.30421537, 50.26581172],
       [27.24327499, 30.48230872],
       [78.26858506, 54.06985999],
       [91.76115346, 63.63264547],
       [11.98819179, 29.02935492],
       [46.96549599, 10.37799855],
       [ 8.46395892,  6.01966828]])

In [78]:
x0 = 25
y0 = 25

omega = np.array([[[0.0, 0.0] for _ in range(N_LANDMARKS + 1)]
                              for _ in range(N_LANDMARKS + 1)])

xi = np.array([[0.0, 0.0] for _ in range(N_LANDMARKS + 1)])
omega[0, 0, :] = np.array([1.0, 1.0])
xi[0, :] = np.array([x0, y0])

#display(omega)
#display(xi)

# Robot class

In [79]:
class Robot:
    def __init__(self,
                 world_size=WORLD_SIZE,
                 measurement_range=MEASUREMENT_RANGE,
                 motion_noise=MOTION_NOISE,
                 measurement_noise=MEASUREMENT_NOISE):

        self.world_size = world_size
        self.measurement_range = measurement_range
        self.motion_noise = motion_noise
        self.measurement_noise = measurement_noise
        self.x = x0
        self.y = y0

    def move(self, dx, dy):
        x = self.x + dx + uniform(-1, 1)*self.motion_noise
        y = self.y + dy + uniform(-1, 1)*self.motion_noise

        self.x = x
        self.y = y

    def sense(self):
        Z = []
        for i, landmark in enumerate(LANDMARKS):
            dx = landmark[0] - self.x + uniform(-1, 1)*self.measurement_noise
            dy = landmark[1] - self.y + uniform(-1, 1)*self.measurement_noise
            if self.measurement_range < 0.0 or sqrt(dx**2 + dy**2) <= self.measurement_range:
                Z.append([i, dx, dy])
        return Z

    def __repr__(self):
        return 'Robot: [x=%.5f y=%.5f]' % (self.x, self.y)



# Slam algorithm

In [83]:
def slam(i, dx, dy, Z):
    global omega, xi
    omega = np.insert(omega, i + 1, 0, axis=0)
    omega = np.insert(omega, i + 1, 0, axis=1)
    xi = np.insert(xi, i + 1, 0, axis=0)
    for meas in Z:
        j, x, y = meas
        omega[i, i] = omega[i, i] + 1/MEASUREMENT_NOISE
        omega[i, i + j + 2] = omega[i, i + j + 2] - 1/MEASUREMENT_NOISE
        omega[i + j + 2, i] = omega[i + j + 2, i] - 1/MEASUREMENT_NOISE
        omega[i + j + 2, i + j + 2] = omega[i + j + 2, i + j + 2] + 1/MEASUREMENT_NOISE
        xi[i, :] = xi[i, :] - np.array([x, y])/MEASUREMENT_NOISE
        xi[i + j + 2, :] = xi[i + j + 2, :] + np.array([x, y])/MEASUREMENT_NOISE
    omega[i, i] = omega[i, i] + 1/MOTION_NOISE
    omega[i + 1, i + 1] = omega[i + 1, i + 1] + 1/MOTION_NOISE
    omega[i + 1, i] = omega[i + 1, i] - 1/MOTION_NOISE
    omega[i, i + 1] = omega[i, i + 1] - 1/MOTION_NOISE
    xi[i, :] = xi[i, :] - np.array([dx, dy])/MOTION_NOISE
    xi[i + 1, :] = xi[i + 1, :] + np.array([dx, dy])/MOTION_NOISE
    mu_x = np.linalg.inv(omega[:, :, 0]).dot(xi[:, 0])
    mu_y = np.linalg.inv(omega[:, :, 1]).dot(xi[:, 1])
    return np.c_[mu_x, mu_y]

dx = [1 for _ in range(50)] + [0 for _ in range(50)]
dx += [-1 for _ in range(50)] + [0 for _ in range(50)]
dy = [0 for _ in range(50)] + [1 for _ in range(50)]
dy += [0 for _ in range(50)] + [-1 for _ in range(50)]

# Animation initialization